# 케라스로 VGGNet 만들기

이 노트북에서 "매우 깊은" 합성곱 신경망 [VGGNet](https://arxiv.org/pdf/1409.1556.pdf)과 비슷한 모델을 훈련하여 옥스포드 꽃 데이터셋을 17개 카테고리로 분류합니다.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/rickiepark/dl-illustrated/blob/master/notebooks/10-3.vggnet_in_keras.ipynb)

#### 동일하게 재현하기 위해 랜덤 시드 값을 지정합니다.

In [1]:
import numpy as np
np.random.seed(42)

#### 라이브러리를 적재합니다.

In [2]:
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.callbacks import TensorBoard 

#### 데이터를 적재하고 전처리합니다.

이미 `oxflower17.npz` 파일을 다운받았다고 가정합니다. 이 파일이 없다면 `10-2.alexnet_in_keras.ipynb` 노트북을 다시 실행하세요.

In [3]:
# 코랩을 사용할 경우 다음 셀을 실행하세요.
!rm oxflower17*
!wget https://bit.ly/31IvwtD -O oxflower17.npz

rm: cannot remove 'oxflower17*': No such file or directory
--2021-02-02 00:10:59--  https://bit.ly/31IvwtD
Resolving bit.ly (bit.ly)... 67.199.248.11, 67.199.248.10
Connecting to bit.ly (bit.ly)|67.199.248.11|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://onedrive.live.com/download?cid=822579D69D2DC3B5&resid=822579D69D2DC3B5!597497&authkey=AGGo9IgYC0QZfXo [following]
--2021-02-02 00:10:59--  https://onedrive.live.com/download?cid=822579D69D2DC3B5&resid=822579D69D2DC3B5!597497&authkey=AGGo9IgYC0QZfXo
Resolving onedrive.live.com (onedrive.live.com)... 13.107.43.13
Connecting to onedrive.live.com (onedrive.live.com)|13.107.43.13|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://evtwow.bl.files.1drv.com/y4mCib8nyCJtEhA2s4GYY7qi42vrTwEdAcdN75jJH3UjySMPPOA1s4_PGF3SFLN8F3THusZgaJN8k4KZyg6A1qsw4XmtFIQ2AOkgg2b8Vls-8saRtKtCbcV2FsNvOHfd4ApOfsh9XdWZi_Mb9nWePA6byCYyui6454htHB_mtnHybQcsoSD-xg895jKsXThfwZbfjtBQxn

In [4]:
import numpy as np

data = np.load('oxflower17.npz')
X = data['X']
Y = data['Y']

#### 신경망 모델을 만듭니다.

In [5]:
model = Sequential()

model.add(Conv2D(64, 3, activation='relu', input_shape=(224, 224, 3)))
model.add(Conv2D(64, 3, activation='relu'))
model.add(MaxPooling2D(2, 2))
model.add(BatchNormalization())

model.add(Conv2D(128, 3, activation='relu'))
model.add(Conv2D(128, 3, activation='relu'))
model.add(MaxPooling2D(2, 2))
model.add(BatchNormalization())

model.add(Conv2D(256, 3, activation='relu'))
model.add(Conv2D(256, 3, activation='relu'))
model.add(Conv2D(256, 3, activation='relu'))
model.add(MaxPooling2D(2, 2))
model.add(BatchNormalization())

model.add(Conv2D(512, 3, activation='relu'))
model.add(Conv2D(512, 3, activation='relu'))
model.add(Conv2D(512, 3, activation='relu'))
model.add(MaxPooling2D(2, 2))
model.add(BatchNormalization())

model.add(Conv2D(512, 3, activation='relu'))
model.add(Conv2D(512, 3, activation='relu'))
model.add(Conv2D(512, 3, activation='relu'))
model.add(MaxPooling2D(2, 2))
model.add(BatchNormalization())

model.add(Flatten())
model.add(Dense(4096, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(4096, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(17, activation='softmax'))

In [6]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 222, 222, 64)      1792      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 220, 220, 64)      36928     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 110, 110, 64)      0         
_________________________________________________________________
batch_normalization (BatchNo (None, 110, 110, 64)      256       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 108, 108, 128)     73856     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 106, 106, 128)     147584    
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 53, 53, 128)       0

#### 모델을 설정합니다.

In [7]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

#### 텐서보드를 설정합니다.

In [8]:
tensorbrd = TensorBoard('logs/vggnet')

#### 훈련!

In [9]:
model.fit(X, Y, batch_size=64, epochs=250, verbose=1, validation_split=0.1, shuffle=True, callbacks=[tensorbrd])

Epoch 1/250
20/20 [==============================] - 29s 760ms/step - loss: 3.3612 - accuracy: 0.1145 - val_loss: 11.1556 - val_accuracy: 0.1029
Epoch 2/250
20/20 [==============================] - 13s 657ms/step - loss: 2.7593 - accuracy: 0.1857 - val_loss: 4.8337 - val_accuracy: 0.1029
Epoch 3/250
20/20 [==============================] - 13s 668ms/step - loss: 2.4863 - accuracy: 0.2205 - val_loss: 2.9562 - val_accuracy: 0.1324
Epoch 4/250
20/20 [==============================] - 14s 684ms/step - loss: 2.2283 - accuracy: 0.2849 - val_loss: 2.2321 - val_accuracy: 0.2132
Epoch 5/250
20/20 [==============================] - 14s 696ms/step - loss: 2.2600 - accuracy: 0.3082 - val_loss: 3.4629 - val_accuracy: 0.0956
Epoch 6/250
20/20 [==============================] - 14s 717ms/step - loss: 2.2498 - accuracy: 0.2921 - val_loss: 2.5904 - val_accuracy: 0.1471
Epoch 7/250
20/20 [==============================] - 15s 738ms/step - loss: 2.0172 - accuracy: 0.2937 - val_loss: 2.2287 - val_accuracy